In [1]:
import json
import ujson
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

import matplotlib.pyplot as plt
% matplotlib inline

In [157]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import coverage_error

def accuracy(model, X, y, features):
    y_true_train = dft['interest_level']
    y_true_valid = dfv['interest_level']
    y_true_test  = dftest['interest_level']
    
    y_pred_train = model.predict(dft[features])
    y_pred_valid = model.predict(dfv[features])
    y_pred_test  = model.predict(dftest[features])
        
    # F1 score
    # print ('F1 score with macro averaging    = ', round(f1_score(y_true, y_pred, average='macro'),2))
    # print ('F1 score with micro averaging    = ', round(f1_score(y_true, y_pred, average='micro'),2))
    # print ('F1 score with weighted averaging = ', round(f1_score(y_true, y_pred, average='weighted'),2))
    
    # Score value
    scores = model.score(X,y)
    print ('Training score value = ', round(scores,2))
    
       
    # Cross Validation scores
    # scores = model_selection.cross_val_score(model, X, y, cv=3, scoring='accuracy')
    # print ('Cross validation score = ', round(scores,2))

In [3]:
training= pd.read_json('rent_hop_train.json')

In [4]:
training['index'] = training.index

In [14]:
# training.head(3)

In [6]:
print (training.columns.values)

['bathrooms' 'bedrooms' 'building_id' 'created' 'description'
 'display_address' 'features' 'interest_level' 'latitude' 'listing_id'
 'longitude' 'manager_id' 'photos' 'price' 'street_address' 'index']


In [7]:
reorder_cols = ['index', 'building_id','interest_level', 'price', 'bathrooms', 'bedrooms', 'features', 'description', 
 'display_address',  'street_address', 'latitude', 'longitude', 'photos', 'manager_id','created','listing_id']

In [8]:
training = training[reorder_cols]

In [17]:
#print (training.columns.values)

In [9]:
training.head(3)

,index,building_id,interest_level,price,bathrooms,bedrooms,features,description,display_address,street_address,latitude,longitude,photos,manager_id,created,listing_id
10,10,53a5b119ba8f7b61d4e010512e0dfc85,medium,3000,1.5,3,[],A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,792 Metropolitan Avenue,40.7145,-73.9425,[https://photos.renthop.com/2/7211212_1ed4542e...,5ba989232d0489da1b5f2c45f6688adc,2016-06-24 07:54:24,7211212
10000,10000,c5c8a357cba207596b04d1afd1e4f130,low,5465,1.0,2,"[Doorman, Elevator, Fitness Center, Cats Allow...",,Columbus Avenue,808 Columbus Avenue,40.7947,-73.9667,[https://photos.renthop.com/2/7150865_be3306c5...,7533621a882f71e25173b27e3139d83d,2016-06-12 12:19:27,7150865
100004,100004,c3ba40552e2120b0acfc3cb5730bb2aa,high,2850,1.0,1,"[Laundry In Building, Dishwasher, Hardwood Flo...","Top Top West Village location, beautiful Pre-w...",W 13 Street,241 W 13 Street,40.7388,-74.0018,[https://photos.renthop.com/2/6887163_de85c427...,d9039c43983f6e564b1482b273bd7b01,2016-04-17 03:26:41,6887163


In [13]:
training.isnull().sum()

index              0
building_id        0
interest_level     0
price              0
bathrooms          0
bedrooms           0
features           0
description        0
display_address    0
street_address     0
latitude           0
longitude          0
photos             0
manager_id         0
created            0
listing_id         0
dtype: int64

### Define classification models and their features

#### Split dataset into train (dft), valid (dfv), test (dftest)

In [18]:
print (training.columns.values)

['index' 'building_id' 'interest_level' 'price' 'bathrooms' 'bedrooms'
 'features' 'description' 'display_address' 'street_address' 'latitude'
 'longitude' 'photos' 'manager_id' 'created' 'listing_id']


In [20]:
from sklearn.model_selection import train_test_split as split
df, dftest = split(training, test_size=0.10, random_state=100)
dft, dfv = split(df, test_size = 0.2, random_state = 100)

In [22]:
dft.head(3)

,index,building_id,interest_level,price,bathrooms,bedrooms,features,description,display_address,street_address,latitude,longitude,photos,manager_id,created,listing_id
69840,69840,4a8bc449d73570bf5eb780c1f0b81313,low,3150,1.0,2,"[Doorman, Elevator, Laundry in Building, Hardw...",WONDERFUL Two Bedroom on Roosevelt Island. Ver...,Main St.,560 Main St.,40.7622,-73.9495,[https://photos.renthop.com/2/7025973_7bfe46b3...,39af186286605963b1d75543e1492c61,2016-05-17 14:42:01,7025973
87053,87053,9f60193f1599b23e425e9ebe4e76b0e8,low,2800,1.0,1,[Pre-War],15 Stanton Stfloor-through studio. 750 sq ft$2...,Stanton Street,15 Stanton Street,40.7224,-73.9923,[https://photos.renthop.com/2/6816174_15d873d7...,c70dac1e7841801c3f8257de18f21561,2016-04-02 05:44:35,6816174
94803,94803,ebbc2f5e5bba7672cbbfe3d12e25053a,low,4195,1.0,2,"[Elevator, Pre-War, Laundry in Building, Dishw...","Prewar Perfection. Sunny, beautiful two bedro...",231 West 21st Street,231 West 21st Street,40.7436,-73.9978,[https://photos.renthop.com/2/6893990_e41095b1...,383f3c86390d40627c14343949cf1d09,2016-04-19 03:28:43,6893990


In [16]:
# rfr = Random Forest Classifier - decision stumps that narrow down interest level path
rfr_features = ['price', 'bathrooms', 'bedrooms']

In [24]:
# text = Text Analyzer - understand key words that predict interest
text_features = ['features', 'description']

In [25]:
# nn = Nearest Neighbor to understand patterns in interest level as it pertains to geographical location
# Need tp create tfidf for display address - choosing this instead of street address....assuming door number is not important

nn_features = ['latitude', 'longitude', 'tfidf_display_address']


In [26]:
# lr = Logistic Regression on RGB pixel intensity
lr_features = ['photos']

In [27]:
target = ['interest_level']

##### Unused features = ['created', 'manager_id', 'listing_id', 'building_id', 'street_address' ]

#### Build individual models

In [52]:
from sklearn import model_selection

# rfr
from sklearn.ensemble import RandomForestClassifier

#text - TFIDF transformer
from sklearn.feature_extraction.text import TfidfVectorizer

# KNN
from sklearn.neighbors import KNeighborsClassifier

# LR
from sklearn.linear_model import LogisticRegression

# Stacking
from mlxtend.classifier import StackingClassifier


In [29]:
# load classifiers objects

rfr = RandomForestClassifier(random_state=1)
nn = KNeighborsClassifier(n_neighbors=1)
lr = LogisticRegression()

### Define stacking classifier

In [30]:
# stack1 = StackingClassifier(classifiers = [clf1, clf3], meta_classifier=lr)

#### Trial with RFR

In [164]:
training.shape[0]

49352

In [35]:
X_rfr = dft[rfr_features]
y = dft['interest_level']

In [169]:
X_full = training[rfr_features]
y_full = training[target]
rfr1 = RandomForestClassifier(random_state = 100, min_samples_leaf = 1)
rfr1.fit(X_full, y_full)
accuracy(rfr1, X_full, y_full, rfr_features)

C:\Users\joisi\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Training score value =  0.73


In [36]:
rfr = rfr.fit(X_rfr, y)

In [50]:
accuracy(rfr, X_rfr, y)
accuracy(rfr, dfv[rfr_features], dfv[target])
accuracy(rfr, dftest[rfr_features], dftest[target])

Score value [0,1]  =  0.73
Score value [0,1]  =  0.68
Score value [0,1]  =  0.7


#### Playing with TFIDF 

In [145]:
from sklearn.feature_extraction.text import CountVectorizer
import operator

cv = CountVectorizer()

train_set = ("Jaisimha is my name", "I love Algorithms")
test_set  = ("Abrar is an employee of mine", "He is a Data Science consultant")

cv.fit_transform(train_set)
vocab = cv.vocabulary_

print ("Unsorted Vocabulary  : ", vocab)
print ("")
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1),reverse=False)
print ("Sorted Vocabulary  : ", sorted_vocab)




Unsorted Vocabulary  :  {'my': 4, 'love': 3, 'is': 1, 'jaisimha': 2, 'name': 5, 'algorithms': 0}

Sorted Vocabulary  :  [('algorithms', 0), ('is', 1), ('jaisimha', 2), ('love', 3), ('my', 4), ('name', 5)]


In [146]:
freq_term_matrix = cv.transform(test_set)
print (freq_term_matrix.todense())


[[0 1 0 0 0 0]
 [0 1 0 0 0 0]]


In [147]:
from sklearn.feature_extraction.text import TfidfTransformer

ttfidf = TfidfTransformer(norm='l2')
ttfidf.fit(freq_term_matrix)

print ("IDF : ", ttfidf.idf_)

IDF :  [ 2.09861229  1.          2.09861229  2.09861229  2.09861229  2.09861229]


#### Trial with TFIDF as features and predict interest_level using Logistic classification

In [96]:
# tfidf = TfidfVectorizer(analyzer='word', norm='l2', sublinear_tf=False, lowercase=True)

In [97]:
# tfidf.fit(dft['description'])

In [98]:
# tfidf.transform(df['description'])

In [99]:
# dft['desc_tfidf'] = tfidf.vocabulary

In [100]:
# print (dft['desc_tfidf'].shape, dft[target].shape)

In [84]:
X_lr = dft['desc_tfidf']
y = dft[target]

In [85]:
lr = lr.fit(X_lr,y)

ValueError: setting an array element with a sequence.

In [49]:
accuracy(lr, X_lr, y)
accuracy(lr, dfv[rfr_features], dfv[target])
accuracy(lr, dftest[rfr_features], dftest[target])

Score value [0,1]  =  0.69
Score value [0,1]  =  0.69
Score value [0,1]  =  0.71


#### ^^^ Logistic regression as its own single classifier does bad just like Linear Regression 

In [232]:
# wrongs = Final[Final['predicted_clf1'] != Final['interest_level']].count()['index']
# acc = wrongs*1./Final.shape[0]
# total = Final.shape[0]
# print (wrongs, total, round(float(1-(wrongs/total)),2))